In [12]:
%load_ext autoreload
%autoreload 2

import os
import sys

import numpy as np
import pandas as pd
import yaml
from itertools import chain, product, combinations
import torch

from time import time as tt

sys.path.append("../../../")
from gnn4itk_cf.stages.data_reading.models.trackml_utils import *

from gnn4itk_cf.stages.data_reading.data_reading_stage import EventReader
from gnn4itk_cf.stages.data_reading.models.trackml_reader import TrackMLReader

from gnn4itk_cf.stages.graph_construction.models.metric_learning import MetricLearning
from gnn4itk_cf.stages.edge_classifier.models.filter import Filter
from gnn4itk_cf.stages.edge_classifier import InteractionGNN

from gnn4itk_cf.stages.graph_construction.utils import handle_weighting
from gnn4itk_cf.stages.graph_construction.models.utils import graph_intersection, build_edges
from gnn4itk_cf.stages.graph_construction.utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
config_file = "trackml_reader_config.yaml"
config = yaml.load(open(config_file, "r"), Loader=yaml.FullLoader)

In [14]:
#reader = TrackMLReader.infer(config)

In [15]:
input_dir = "/workspaces/train100events/results/feature_store/trainset"
sample = torch.load(os.path.join(input_dir, "event000001001-graph.pyg"), map_location="cpu")
sample

Data(cell_val=[93680], geta=[93680], weight=[93680], region=[93680], lx=[93680], lphi=[93680], module_index=[93680], x=[93680], r=[93680], gphi=[93680], hit_id=[93680], lz=[93680], z=[93680], cell_count=[93680], phi=[93680], y=[93680], ly=[93680], leta=[93680], eta=[93680], track_edges=[2, 68512], particle_id=[68512], radius=[68512], nhits=[68512], pt=[68512], config=[1], event_id='000001001')

In [16]:
with open("/workspaces/acorn/examples/Example_3/metric_learning_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_ML = MetricLearning(config)
with open("/workspaces/acorn/examples/Example_3/filter_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_filter = Filter(config)
with open("/workspaces/acorn/examples/Example_3/gnn_train.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model_gnn = InteractionGNN(config)

In [17]:
model_ML.setup(stage="predict")
dataloaders = model_ML.predict_dataloader()
#dataloaders[2].dataset.__dict__
model_filter.setup('predict')
model_gnn.setup('predict')

Loaded 80 training events, 10 validation events and 10 testing events
Defining figures of merit
Defining figures of merit


In [18]:
%%time
model_ML = model_ML.to("cuda")
model_filter = model_filter.to("cuda")
model_gnn = model_gnn.to("cuda")
for batch in dataloaders[2]:
    batch = batch.to("cuda")
    with torch.no_grad():
        embedding = model_ML.apply_embedding(batch)
     
    batch.edge_index = build_edges(
        query=embedding, database=embedding, indices=None, r_max=0.1, k_max=10, backend="FRNN"
    )
    R = batch.r**2 + batch.z**2
    flip_edge_mask = R[batch.edge_index[0]] > R[batch.edge_index[1]]
    batch.edge_index[:, flip_edge_mask] = batch.edge_index[:, flip_edge_mask].flip(0)
    #out = model_filter(batch)   
    #preds = torch.sigmoid(out) > 0.5
    #update batch.edge_index
    out = model_gnn(batch)
    preds = torch.sigmoid(out) > 0.5
    print(out)
    sample=batch



tensor([-0.0329, -0.0412, -0.1953,  ...,  0.4911,  0.4876,  0.4333],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([ 0.3679,  0.0570,  0.0410,  ..., -0.1526,  0.0040,  0.0806],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([ 0.1463, -0.0067,  0.0218,  ...,  0.1621,  0.2134,  0.1807],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([ 0.0312, -0.0760, -0.0247,  ...,  0.0831,  0.1865, -0.0918],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([ 0.1532,  0.0218,  0.0165,  ..., -0.0594, -0.1280, -0.1287],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([0.0107, 0.0271, 0.0598,  ..., 0.4425, 0.1881, 0.1814], device='cuda:0',
       grad_fn=<SqueezeBackward1>)
tensor([ 0.2371,  0.1997,  0.1279,  ..., -0.3052,  0.0663,  0.0662],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.0847, -0.0056,  0.0223,  ...,  0.4740,  0.4753,  0.2840],
       device='cuda:0', grad_fn=<SqueezeBackward1>)
tensor([-0.1496, -0.0896, -0.0814,  ..

In [19]:
sample=sample.to_dict()

In [20]:
sample

{'cell_val': tensor([0.2916, 0.3245, 0.2921,  ..., 1.0000, 1.0000, 1.0000], device='cuda:0',
        dtype=torch.float64),
 'geta': tensor([-2.0914, -1.4405, -2.0914,  ...,  0.0673,  0.0673,  0.0673],
        device='cuda:0', dtype=torch.float64),
 'weight': tensor([1.8809e-05, 1.8770e-05, 2.7282e-05,  ..., 2.4526e-05, 1.6866e-05,
         3.4333e-05], device='cuda:0', dtype=torch.float64),
 'region': tensor([1., 1., 1.,  ..., 6., 6., 6.], device='cuda:0', dtype=torch.float64),
 'lx': tensor([0.5000, 1.0000, 0.5000,  ..., 1.2000, 1.2000, 1.2000], device='cuda:0',
        dtype=torch.float64),
 'lphi': tensor([0.8442, 0.8442, 0.8442,  ..., 1.5593, 1.5593, 1.5593], device='cuda:0',
        dtype=torch.float64),
 'module_index': tensor([    0,     1,     2,  ..., 18717, 18718, 18726], device='cuda:0'),
 'x': tensor([ -58.8380, -119.9720, -112.5270,  ..., -710.8110, -648.0540,
         -802.9250], device='cuda:0', dtype=torch.float64),
 'r': tensor([0.0597, 0.1200, 0.1149,  ..., 0.9317, 0.

In [21]:
export_output = torch.onnx.export(model_filter, sample,"filter.onnx")

TypeError: new(): invalid data type 'str'

In [ ]:
# # Positives
# edge_positive = preds.sum().float()

# # Signal true & signal tp
# sig_true = outputs["target_truth"].sum().float()
# sig_true_positive = (outputs["target_truth"].bool() & preds).sum().float()
# background_true_positive = (outputs["all_truth"].bool() & preds).sum().float()
# sig_eff = sig_true_positive / sig_true
# sig_pur = sig_true_positive / edge_positive
# background_pur = background_true_positive / edge_positive

In [1]:
from frnn import _C